<a href="https://colab.research.google.com/github/FilipeFariaDias/information-retrival-using-pyterrier/blob/main/Information_Retrieval_with_PyTerrier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Apresentação

O [Terrier](http://terrier.org/) é uma ferramenta open source de Recuperação de Informações desenvolvida pela Universidade de Glasgow. Além das funcionalidades básicas de indexação e consulta, o Terrier implementa várias técnicas do estado da arte de melhoria de desempenho. Nessa disciplina, iremos usar o PyTerrier, uma API em Python para o Terrier.
Documentação em https://pyterrier.readthedocs.io/_/downloads/en/latest/pdf/


## Instalação do PyTerrier


In [ ]:
pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip install pyserini
!pip install python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
# !pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 72.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00
  Created wheel for n

## Init

É necessário executar `pt.init()` para poder usar as funções do PyTerrier

Argumentos Opcionais:
 - `version` - terrier IR version e.g. "5.2"
 - `mem` - megabytes allocated to java e.g. "4096"
 - `packages` - external java packages for Terrier to load e.g. ["org.terrier:terrier.prf"]
 - `logging` - logging level for Terrier. Defaults to "WARN", use "INFO" or "DEBUG" for more output.


In [ ]:
import pyterrier as pt
import pandas as pd
import pyserini
import json
import os
import hashlib
import shutil
if not pt.started():
  pt.init()
import onir_pt
from pprint import pprint
from sklearn.model_selection import train_test_split

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


#Getting the datasets

In [ ]:
!!pip install wget

['Requirement already satisfied: wget in /usr/local/lib/python3.10/dist-packages (3.2)']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
shutil.rmtree('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/trec_docs')
os.makedirs('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/trec_docs')

Mounted at /content/drive


In [ ]:
DATASET_PATH = "/content/drive/My Drive/data/training_data/training_data/"

TOPICS_PATH = "/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics.txt"
TEST_TOPICS_PATH = "/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics_test.txt"
TRAIN_TOPICS_PATH = "/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics_train.txt"

QRELS_PATH = "/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels.txt"
TRAIN_QRELS_PATH = "/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels_train.txt"
TEST_QRELS_PATH = "/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels_test.txt"

files = pt.io.find_files("/content/drive/My Drive/data/training_data/training_data/CT json")
DOCS_PATH = os.path.join(DATASET_PATH, "CT json")
DEV_PATH = os.path.join(DATASET_PATH, "dev.json")
TRAIN_PATH = os.path.join(DATASET_PATH, "train.json")
TREC_DOCS_PATH = '/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/trec_docs'

gh_files = pt.io.find_files("/content/drive/My Drive/data/GH95/docs")

In [ ]:
with open(TRAIN_PATH) as f:
  train_files = json.load(f)

with open(DEV_PATH) as f:
  dev_files = json.load(f)

In [ ]:
docs_data = []
for file_ in files:
  if '.json' in file_:
    docs_data.append(json.load(open(file_, 'r')))

In [ ]:
columns = ["docno", "Type", "Section_id", "Primary_id", "Secondary_id", "Statement", "Label", "Primary_evidence_index", "Secondary_evidence_index"]

topics = pt.io.read_topics(TOPICS_PATH)
# train_topics = pt.io.read_topics(TRAIN_TOPICS_PATH)
test_topics = pt.io.read_topics(TEST_TOPICS_PATH)

qrels = pt.io.read_qrels(QRELS_PATH)
train_qrels = pt.io.read_qrels(TRAIN_QRELS_PATH)
test_qrels = pt.io.read_qrels(TEST_QRELS_PATH)

dev_df = pd.DataFrame.from_dict(dev_files, orient='index').reset_index()
train_df = pd.DataFrame.from_dict(train_files, orient='index').reset_index()
# train_df.columns = columns
docs_df = pd.DataFrame(docs_data)

In [ ]:
train, test = train_test_split(docs_df, test_size=0.2)

In [ ]:
dev_df

,index,Type,Section_id,Primary_id,Statement,Label,Primary_evidence_index,Secondary_id,Secondary_evidence_index
0,1adc970c-d433-44d0-aa09-d3834986f7a2,Single,Results,NCT00066573,there is a 13.2% difference between the result...,Contradiction,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",NaN,NaN
1,6b9162d0-0816-46d4-81af-c60028dcc63b,Comparison,Eligibility,NCT00425854,Patients with significantly elevated ejection ...,Contradiction,[15],NCT01224678,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,0b6cc8e3-69ee-4a91-b93d-2ad3fddce65f,Comparison,Adverse Events,NCT02273973,a significant number of the participants in th...,Contradiction,"[5, 18]",NCT00281697,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,cc1f712a-2116-4e40-9810-f315e3fa5ff8,Single,Results,NCT00593346,the primary trial does not report the PFS or o...,Entailment,"[0, 1, 2, 3]",NaN,NaN
4,904061c0-14fa-4f13-9118-9a41e24fa8eb,Single,Eligibility,NCT02340221,Prior treatment with fulvestrant or with a pho...,Contradiction,[13],NaN,NaN
...,...,...,...,...,...,...,...,...,...
195,d310ec4e-993e-4827-8dc5-9aca053972db,Comparison,Intervention,NCT00688909,The the primary trial intervention involves on...,Contradiction,"[0, 1, 2]",NCT00450723,"[0, 1, 2]"
196,a5617ae4-05a3-42d0-9e14-141de5f8c010,Comparison,Adverse Events,NCT00258960,the secondary trial reported 1 single case of ...,Entailment,"[0, 1, 2, 3, 4, 5, 6, 7, 8]",NCT00121992,"[0, 9, 14, 23]"
197,42d1fcd3-8faa-4065-bbba-42cc90ab67fb,Comparison,Results,NCT00856492,the secondary trial and the primary trial do n...,Entailment,"[0, 1, 2, 3]",NCT00009945,"[0, 1, 2, 3]"
198,d01fda83-5dc8-4ad5-92b8-7553dabd7046,Single,Results,NCT00428922,the outcome measurement of the primary trial i...,Entailment,"[0, 1]",NaN,NaN


In [ ]:
train_df

,index,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction,"[0, 1, 2, 3, 4, 5]","[0, 1, 2, 3, 4, 5]"
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction,"[18, 22, 23, 24]",NaN
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment,"[0, 3]","[0, 7, 8, 9, 10]"
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction,"[0, 1, 3, 4, 5]",NaN
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction,"[0, 1, 2, 3, 4, 5, 6, 7]","[0, 1, 2, 3, 4, 5, 6]"
...,...,...,...,...,...,...,...,...,...
1695,f37774f4-db96-4aa6-b3a1-626953faeecf,Comparison,Eligibility,NCT00213980,NCT02536339,"Adequate blood, kidney, and hepatic function a...",Entailment,"[0, 1, 2, 3, 4, 5]","[0, 7]"
1696,4fef4cdf-53bf-4239-9d31-4710fd3edc6f,Single,Results,NCT01605396,NaN,The Ridaforolimus + Dalotuzumab + Exemestane g...,Contradiction,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",NaN
1697,331affb2-f8e9-4a55-ac4c-62d2ecc4f80b,Single,Intervention,NCT00181363,NaN,The only difference between the interventions ...,Entailment,"[0, 1, 2, 3, 4, 5]",NaN
1698,a577e819-c928-4217-8743-f4809e852919,Single,Eligibility,NCT00834678,NaN,Patients must have a white blood cell count ab...,Entailment,[18],NaN


In [ ]:
docs_df

,Clinical Trial ID,Intervention,Eligibility,Results,Adverse Events
0,NCT00001832,"[INTERVENTION 1: , Abl Cells IV + Cyclophosp...","[INCLUSION CRITERIA, Patients must have eval...","[Outcome Measurement: , Clinical Response, ...","[Adverse Events 1:, Total: 0/3 (0.00%), Ly..."
1,NCT00003199,"[INTERVENTION 1: , TX/Maintenance Therapy fo...","[Inclusion Criteria:, Patients with inflamma...","[Outcome Measurement: , Event-free Survival,...","[Adverse Events 1:, Total: 2/50 (4.00%), P..."
2,NCT00003404,"[INTERVENTION 1: , Adjuvant Radiotherapy, ...","[DISEASE CHARACTERISTICS:, Histologically pr...","[Outcome Measurement: , Local Recurrence Rat...","[Adverse Events 1:, Total: 0/46 (0.00%)]"
3,NCT00003782,"[INTERVENTION 1: , Arm 1: Doxorubicin + Cycl...","[DISEASE CHARACTERISTICS:, Histologically co...","[Outcome Measurement: , Overall Survival, ...","[Adverse Events 1:, Total: 66/1748 (3.78%), ..."
4,NCT00003830,"[INTERVENTION 1: , Arm I:Sentinel Node Resec...","[DISEASE CHARACTERISTICS:, Resectable invasi...","[Outcome Measurement: , Morbidity - Number o...","[Adverse Events 1:, Total: 9/2788 (0.32%), ..."
...,...,...,...,...,...
994,NCT03719677,"[INTERVENTION 1: , Habit Development Interve...","[Inclusion Criteria:, English speaking, Di...","[Outcome Measurement: , Self Reported Behavi...","[Adverse Events 1:, Total: 0/7 (0.00%)]"
995,NCT03765996,"[INTERVENTION 1: , Decongestive Physiotherap...","[Inclusion Criteria:, Patients who had unila...","[Outcome Measurement: , Change of the Limb V...","[Adverse Events 1:, Total: 0/18 (0.00%), Adv..."
996,NCT04030104,"[INTERVENTION 1: , IUS Alone, IUS alone imag...","[Inclusion Criteria:, One analyzable mass pe...","[Outcome Measurement: , Gain in Specificity ...","[Adverse Events 1:, Total: 2/480 (0.42%), ..."
997,NCT04080297,"[INTERVENTION 1: , 100 mg Q-122, Dosage wa...","[Inclusion Criteria:, Be a female of any rac...","[Outcome Measurement: , Adverse Event (AE) R...","[Adverse Events 1:, Total: 0/10 (0.00%), B..."



# PyTerrier Indexing

More examples in (https://github.com/terrier-org/pyterrier).



## Creating files in TREC format

We used the task 7 from SemEval 2023 data to create the topics, QRELS and the TREC files.

The TREC files were created at different levels to find which level of indexing results in a better retrieving result.

At document level each TREC file is a different clinical trial. At section leval each file is a section of a clinical trial. And at passage level each file is a evidence of the clinical trial.

The problem is that we can't know for sure which evidence is a better response for a certain hypothesis because the task does'nt provide the topics, qrels or even the scores from each evidence to know the right answer. So the data is not suitable for passage/information retrieval task.

In [ ]:
import json

with open(DEV_PATH) as json_file:
    dev = json.load(json_file)

with open(TRAIN_PATH) as json_file:
    train_data = json.load(json_file)

# Example instance
# keys = list(train_data.keys())
# print(dev[keys[0]])
# print(len(keys))

In [ ]:
def get_evidence_list(rct_path, section_id=None):
  evidence_file = open(rct_path, 'r')
  evidence_data = json.load(evidence_file)
  evidence_file.close()
  del evidence_file
  if section_id is not None:
    return evidence_data[section_id]
  return evidence_data

In [ ]:
topic_template = """
<top>
  <num>%d</num>
  <title>%s</title>
  <desc>%s</desc>
  <narr>%s</narr>
</top>\n
"""

doc_template = """<DOC>
  <DOCNO>%s</DOCNO>
  <DOCID>%s</DOCID>
  <TEXT>%s</TEXT>
</DOC>
"""

docid_template = "%s-%s-%d"
id_template = "%s-%s"

qrels_template = "%d 0 %s %d\n"

In [ ]:
# num = 0
# train_count = 0
# test_count = 0
# for k in keys:
#   # print(k)
#   num += 1
#   rct = train_data[k]['Primary_id']
#   section = train_data[k]['Section_id']
#   evidence_list = get_evidence_list(os.path.join(DOCS_PATH, rct + '.json'), section)
#   hypothesis = train_data[k]['Statement']
#   train_count += 1 if rct in train.values else 0
#   test_count += 1 if rct in test.values else 0
#   # for j, evidence in enumerate(evidence_list):
#   #   pprint(train_data[k].keys())
#   # break
# print(f'train: {train_count}')
# print(f'test: {test_count}')
# print(f'num: {num}')

In [ ]:
def get_section_data(rct_path, section):
  evidence_file = open(rct_path, 'r')
  evidence_data = json.load(evidence_file)
  evidence_file.close()
  del evidence_file
  return evidence_data[section]

In [ ]:
def get_sections(rct_path):
  evidence_file = open(rct_path, 'r')
  evidence_data = json.load(evidence_file)
  evidence_file.close()
  del evidence_file
  return list(evidence_data.keys())


In [ ]:
keys = list(train_data.keys())

In [ ]:
all_files = [file_.split('/')[-1][:-5] for file_ in files]

**Document files**

Creating TREC files using all clinical trials

In [ ]:
for file_ in files:
  if '.json' in file_:
    content = json.dumps(json.load(open(file_, 'r')))
    filename_ = file_.split('/')[-1][:-5]
    doc_tmp = doc_template % (filename_, filename_, content)
    with open(os.path.join(TREC_DOCS_PATH, filename_ + '.sgml'), 'w') as f:
      f.write(doc_tmp)

In [ ]:
struct = list()
dict_doc = {}
trec_doc = ""
content = ""
qrels_content = ""
train_content = ""
test_content = ""
train_qrels_content = ""
test_qrels_content = ""

num = 0
for k in keys:
  num += 1
  rct = train_data[k]['Primary_id']
  hypothesis = train_data[k]['Statement']
  content += (topic_template % (num, hypothesis, hypothesis, hypothesis))

  train_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in train_df.values else ""
  # test_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in test.values else ""

  for file_ in all_files:
    if train_data[k]['Type'] == 'Single':
      qrels_content += (qrels_template % (num, file_, int(file_ in train_data[k]['Primary_id'])))
      train_qrels_content += (qrels_template % (num, file_, int(file_ in train_data[k]['Primary_id']))) if rct in train_df.values else ""
      # test_qrels_content += (qrels_template % (num, file_, int(file_ in train_data[k]['Primary_id']))) if rct in test.values else ""
    else:
      train_qrels_content += (qrels_template % (num, file_, int(file_ in train_data[k]['Primary_id'] or file_ in train_data[k]['Secondary_id']))) if rct in train_df.values else ""
      # test_qrels_content += (qrels_template % (num, file_, int(file_ in train[k]['Primary_id'] or file_ in train[k]['Secondary_id']))) if rct in test.values else ""
      qrels_content += (qrels_template % (num, file_, int(file_ in train_data[k]['Primary_id'] or file_ in train_data[k]['Secondary_id'])))

In [ ]:
keys = list(dev.keys())

In [ ]:
# struct = list()
# content = ""
# qrels_content = ""
# train_content = ""
test_content = ""
# train_qrels_content = ""
test_qrels_content = ""
num = 0
for k in keys:
  num += 1
  rct = dev[k]['Primary_id']
  hypothesis = dev[k]['Statement']
  content += (topic_template % (num, hypothesis, hypothesis, hypothesis))

  # train_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in train.values else ""
  test_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in dev_df.values else ""

  for file_ in all_files:
    if dev[k]['Type'] == 'Single':
      qrels_content += (qrels_template % (num, file_, int(file_ in dev[k]['Primary_id'])))
      # train_qrels_content += (qrels_template % (num, file_, int(file_ in dev[k]['Primary_id']))) if rct in train.values else ""
      test_qrels_content += (qrels_template % (num, file_, int(file_ in dev[k]['Primary_id']))) if rct in dev_df.values else ""
    else:
      # train_qrels_content += (qrels_template % (num, file_, int(file_ in dev[k]['Primary_id'] or file_ in dev[k]['Secondary_id']))) if rct in train.values else ""
      test_qrels_content += (qrels_template % (num, file_, int(file_ in dev[k]['Primary_id'] or file_ in dev[k]['Secondary_id']))) if rct in dev_df.values else ""
      qrels_content += (qrels_template % (num, file_, int(file_ in dev[k]['Primary_id'] or file_ in dev[k]['Secondary_id'])))

**Section files**

Creating TREC files using the sections of the clinical trials

In [ ]:
# struct = list()
# content = ""
# qrels_content = ""
# train_content = ""
# test_content = ""
# train_qrels_content = ""
# test_qrels_content = ""
# num = 0
# for k in keys:
#   num += 1
#   rct = train_data[k]['Primary_id']
#   section_id = train_data[k]['Section_id']
#   section_list = get_sections(os.path.join(DOCS_PATH, rct + '.json'))
#   hypothesis = train_data[k]['Statement']
#   content += (topic_template % (num, hypothesis, hypothesis, hypothesis))
#   train_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in train.values else ""
#   test_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in test.values else ""
#   for section in section_list:
#     docid_tmp = hashlib.sha1((id_template % (rct, section)).encode('utf-8')).hexdigest()
#     qrels_content += (qrels_template % (num, docid_tmp, int(section_id in section)))
#     train_qrels_content += (qrels_template % (num, docid_tmp, int(section_id in section))) if rct in train.values else ""
#     test_qrels_content += (qrels_template % (num, docid_tmp, int(section_id in section))) if rct in test.values else ""

#     if docid_tmp not in struct:
#       doc_tmp = doc_template % (docid_tmp, docid_tmp, get_section_data(os.path.join(DOCS_PATH, rct + '.json'), section))
#       with open(os.path.join(TREC_DOCS_PATH, docid_tmp + '.sgml'), 'w') as f:
#         f.write(doc_tmp)
#       struct.append(docid_tmp)

#   if train_data[k]['Type'] == 'Comparison':
#     rct = train_data[k]['Secondary_id']
#     section_list = get_sections(os.path.join(DOCS_PATH, rct + '.json'))

#     for section in section_list:
#       docid_tmp = hashlib.sha1((id_template % (rct, section)).encode('utf-8')).hexdigest()
#       qrels_content += (qrels_template % (num, docid_tmp, int(section_id in section)))
#       train_qrels_content += (qrels_template % (num, docid_tmp, int(section_id in section))) if rct in train.values else ""
#       test_qrels_content += (qrels_template % (num, docid_tmp, int(section_id in section))) if rct in test.values else ""

#       if docid_tmp not in struct:
#         doc_tmp = doc_template % (docid_tmp, docid_tmp, get_section_data(os.path.join(DOCS_PATH, rct + '.json'), section))
#         with open(os.path.join(TREC_DOCS_PATH, docid_tmp + '.sgml'), 'w') as f:
#           f.write(doc_tmp)
#         struct.append(docid_tmp)

**Passage files**

Creating TREC files using the evidences of clinical trials.  

In [ ]:
# struct = list()
# dict_doc = {}
# trec_doc = ""
# content = ""
# qrels_content = ""
# train_content = ""
# test_content = ""
# train_qrels_content = ""
# test_qrels_content = ""
# num = 0
# for k in keys:
#   num += 1
#   rct = train_data[k]['Primary_id']
#   section = train_data[k]['Section_id']
#   evidence_list = get_evidence_list(os.path.join(DOCS_PATH, rct + '.json'), section)
#   hypothesis = train_data[k]['Statement']

#   content += (topic_template % (num, hypothesis, hypothesis, hypothesis))
#   train_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in train_df.values else ""
#   # test_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in test.values else ""
#   for j, evidence in enumerate(evidence_list):
#     docid_tmp = hashlib.sha1((docid_template % (rct, section, j)).encode('utf-8')).hexdigest()
#     train_qrels_content += (qrels_template % (num, docid_tmp, int(j in train_data[k]['Primary_evidence_index']))) if rct in train_df.values else ""
#     # test_qrels_content += (qrels_template % (num, docid_tmp, int(j in train_data[k]['Primary_evidence_index']))) if rct in test.values else ""
#     qrels_content += (qrels_template % (num, docid_tmp, int(j in train_data[k]['Primary_evidence_index'])))
#     # if j in train_data[k]['Primary_evidence_index']:
#     #   qrels_content += (qrels_template % (num, docid_tmp, 1))   #criar qrels somente com valores uteis
#     #   if rct in train.values:
#     #     train_qrels_content += (qrels_template % (num, docid_tmp, 1))
#     #   if rct in test.values:
#     #     test_qrels_content += (qrels_template % (num, docid_tmp, 1))

#     if docid_tmp not in struct:
#       dict_doc[docid_tmp] ={}
#       dict_doc[docid_tmp]['docid']= docid_tmp
#       dict_doc[docid_tmp]['text']= evidence.strip()
#       trec_doc += doc_template % (docid_tmp, docid_tmp, evidence.strip())
#       doc_tmp = doc_template % (docid_tmp, docid_tmp, evidence.strip())
#       with open(os.path.join(TREC_DOCS_PATH, docid_tmp + '.sgml'), 'w') as f:
#         f.write(doc_tmp)
#       struct.append(docid_tmp)

#   if train_data[k]['Type'] == 'Comparison':
#     rct = train_data[k]['Secondary_id']
#     evidence_list = get_evidence_list(os.path.join(DOCS_PATH, rct + '.json'), section)
#     for j, evidence in enumerate(evidence_list):
#       docid_tmp = hashlib.sha1((docid_template % (rct, section, j)).encode('utf-8')).hexdigest()
#       train_qrels_content += (qrels_template % (num, docid_tmp, int(j in train_data[k]['Secondary_evidence_index']))) if rct in train_df.values else ""
#       # test_qrels_content += (qrels_template % (num, docid_tmp, int(j in train_data[k]['Secondary_evidence_index']))) if rct in test.values else ""
#       qrels_content += (qrels_template % (num, docid_tmp, int(j in train_data[k]['Secondary_evidence_index'])))
#       # if j in train_data[k]['Secondary_evidence_index']:
#       #   qrels_content += (qrels_template % (num, docid_tmp, 1))
#       #   if rct in train.values:
#       #     train_qrels_content += (qrels_template % (num, docid_tmp, 1))
#       #   if rct in test.values:
#       #     test_qrels_content += (qrels_template % (num, docid_tmp, 1))

#       if docid_tmp not in struct:
#         dict_doc[docid_tmp] ={}
#         dict_doc[docid_tmp]['docid']= docid_tmp
#         dict_doc[docid_tmp]['text']= evidence.strip()
#         trec_doc += doc_template % (docid_tmp, docid_tmp, evidence.strip())
#         doc_tmp = doc_template % (docid_tmp, docid_tmp, evidence.strip())
#         with open(os.path.join(TREC_DOCS_PATH, docid_tmp + '.sgml'), 'w') as f:
#           f.write(doc_tmp)
#         struct.append(docid_tmp)


In [ ]:
# keys = list(dev.keys())

In [ ]:
# struct = list()
# # content = ""
# # qrels_content = ""
# # train_content = ""
# test_content = ""
# # train_qrels_content = ""
# test_qrels_content = ""
# num = 0
# for k in keys:
#   num += 1
#   rct = dev[k]['Primary_id']
#   section = dev[k]['Section_id']
#   evidence_list = get_evidence_list(os.path.join(DOCS_PATH, rct + '.json'), section)
#   hypothesis = dev[k]['Statement']

#   content += (topic_template % (num, hypothesis, hypothesis, hypothesis))
#   # train_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in train.values else ""
#   test_content += (topic_template % (num, hypothesis, hypothesis, hypothesis)) if rct in dev_df.values else ""
#   for j, evidence in enumerate(evidence_list):
#     docid_tmp = hashlib.sha1((docid_template % (rct, section, j)).encode('utf-8')).hexdigest()
#     # train_qrels_content += (qrels_template % (num, docid_tmp, int(j in dev[k]['Primary_evidence_index']))) if rct in train.values else ""
#     test_qrels_content += (qrels_template % (num, docid_tmp, int(j in dev[k]['Primary_evidence_index']))) if rct in dev_df.values else ""
#     qrels_content += (qrels_template % (num, docid_tmp, int(j in dev[k]['Primary_evidence_index'])))
#     # if j in dev[k]['Primary_evidence_index']:
#     #   qrels_content += (qrels_template % (num, docid_tmp, 1))
#     #   if rct in train.values:
#     #     train_qrels_content += (qrels_template % (num, docid_tmp, 1))
#     #   if rct in test.values:
#     #     test_qrels_content += (qrels_template % (num, docid_tmp, 1))

#     if docid_tmp not in struct:
#       dict_doc[docid_tmp] ={}
#       dict_doc[docid_tmp]['docid']= docid_tmp
#       dict_doc[docid_tmp]['text']= evidence.strip()
#       trec_doc += doc_template % (docid_tmp, docid_tmp, evidence.strip())
#       doc_tmp = doc_template % (docid_tmp, docid_tmp, evidence.strip())
#       with open(os.path.join(TREC_DOCS_PATH, docid_tmp + '.sgml'), 'w') as f:
#         f.write(doc_tmp)
#       struct.append(docid_tmp)

#   if dev[k]['Type'] == 'Comparison':
#     rct = dev[k]['Secondary_id']
#     evidence_list = get_evidence_list(os.path.join(DOCS_PATH, rct + '.json'), section)
#     for j, evidence in enumerate(evidence_list):
#       docid_tmp = hashlib.sha1((docid_template % (rct, section, j)).encode('utf-8')).hexdigest()
#       # train_qrels_content += (qrels_template % (num, docid_tmp, int(j in dev[k]['Secondary_evidence_index']))) if rct in train.values else ""
#       test_qrels_content += (qrels_template % (num, docid_tmp, int(j in dev[k]['Secondary_evidence_index']))) if rct in dev_df.values else ""
#       qrels_content += (qrels_template % (num, docid_tmp, int(j in dev[k]['Secondary_evidence_index'])))
#       # if j in dev[k]['Secondary_evidence_index']:
#       #   qrels_content += (qrels_template % (num, docid_tmp, 1))
#       #   if rct in train.values:
#       #     train_qrels_content += (qrels_template % (num, docid_tmp, 1))
#       #   if rct in test.values:
#       #     test_qrels_content += (qrels_template % (num, docid_tmp, 1))

#       if docid_tmp not in struct:
#         dict_doc[docid_tmp] ={}
#         dict_doc[docid_tmp]['docid']= docid_tmp
#         dict_doc[docid_tmp]['text']= evidence.strip()
#         trec_doc += doc_template % (docid_tmp, docid_tmp, evidence.strip())
#         doc_tmp = doc_template % (docid_tmp, docid_tmp, evidence.strip())
#         with open(os.path.join(TREC_DOCS_PATH, docid_tmp + '.sgml'), 'w') as f:
#           f.write(doc_tmp)
#         struct.append(docid_tmp)


Creating topics, qrels and trec files.
They will be used for indexing and the retrieving methods    

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics_train.txt', 'w') as f:
  f.write(train_content)

with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics_test.txt', 'w') as f:
  f.write(test_content)

with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels_train.txt', 'w') as f:
  f.write(train_qrels_content)

with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels_test.txt', 'w') as f:
  f.write(test_qrels_content)

with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics.txt', 'w') as f:
  f.write(content)

with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels.txt', 'w') as f:
  f.write(qrels_content)

with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/trec_doc.sgml', 'w') as f:
  f.write(trec_doc)

json_doc = json.dumps(dict_doc)
with open('/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/json_doc.json', 'w') as f:
  f.write(json_doc)

In [ ]:
topics = pt.io.read_topics(TOPICS_PATH)
train_topics = pt.io.read_topics(TRAIN_TOPICS_PATH)
test_topics = pt.io.read_topics(TEST_TOPICS_PATH)

qrels = pt.io.read_qrels(QRELS_PATH)
train_qrels = pt.io.read_qrels(TRAIN_QRELS_PATH)
test_qrels = pt.io.read_qrels(TEST_QRELS_PATH)


Transforming the json files to trec files

In [ ]:
# def json_to_trec(json_file, trec_file, docid):
#   with open(json_file) as f:
#     data = json.load(f)

#   with open(trec_file, 'w') as f:
#     f.write("<DOC>\n")
#     f.write(f"<DOCNO>{data['Clinical Trial ID']}</DOCNO>\n")
#     f.write(f"<DOCID>{data['Clinical Trial ID']}</DOCID>\n")
#     f.write(f"<TEXT>{data}</TEXT>")



Creating empty trec files with the same name as the json formatted files

In [ ]:
# json_directory = '/content/drive/MyDrive/Colab Notebooks/CTRs'
# trec_directory = '/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/trec_docs'
# json_ext = '.json'
# trec_ext = '.sgml'

# filenames = [f for f in os.listdir(docs) if f.endswith(json_ext)]

# for filename in filenames:
#   new_filename = os.path.splitext(filename)[0] + trec_ext
#   open(os.path.join(trec_directory, new_filename), 'w').close()


Getting the TREC files


In [ ]:
trec_files = pt.io.find_files("/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/trec_docs")
# docid = 0
# for json_file in files:
#   for trec_file in trec_files:
#     docid+=1
#     if os.path.basename(os.path.splitext(json_file)[0]) == os.path.basename(os.path.splitext(trec_file)[0]):
#       json_to_trec(json_file, trec_file, docid)

# Indexing the TREC formatted files

In [ ]:
INDEX_DIR='/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/index' #diretório aonde o índice vai ficar
indexer = pt.TRECCollectionIndexer(INDEX_DIR,
    # vamos salvar o texto como metadados
    meta= {'docno' : 50, 'text' : 4096},
    # The tags from which to save the text. ELSE is special tag name, which means anything not consumed by other tags.
    meta_tags = {'text' : 'ELSE'},
    verbose=True,
    overwrite=True) #para sobrescrever, caso já tenha um índice com aquele nome
indexref = indexer.index(trec_files)
#Indexando os arquivos -- chamando o método index no objeto TRECCollectionIndexer
index = pt.IndexFactory.of(indexref)

  0%|          | 0/999 s<?, ?files/s]

##Training and reranking models

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='text')

[2023-08-02 15:35:32,242][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2023-08-02 15:35:33,796][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [1.55s]


In [ ]:
pipeline = (pt.BatchRetrieve(index) % 100 # get top 100 results
            >> pt.text.get_text(index, 'text') # fetch the document text
            >> knrm) # apply neural re-ranker


In [ ]:
br = pt.BatchRetrieve(index)
# foo = (pt.BatchRetrieve(index) # get top 100 results
#        >> pt.text.get_text(index, 'text') # fetch the document text
#        >> trained_model)

pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    eval_metrics=["recip_rank", "map", "recall_5", "recall_10", "P.1", "P.5", "P.10", "mrt"]
)

[2023-08-02 15:59:50,339][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 15:59:50,830][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/57500 s<?, ?it/s]

[2023-08-02 16:03:03,009][onir_pt][DEBUG] [finished] batches: [03:12] [57500it] [299.20it/s]


,name,recip_rank,map,recall_5,recall_10,P.1,P.5,P.10,mrt
0,DPH,0.218014,0.192459,0.228235,0.279118,0.161765,0.053294,0.033353,375.448365
1,DPH >> VBERT,0.021444,0.017253,0.013235,0.030882,0.002353,0.003765,0.004176,486.415703


In [ ]:
pipeline.fit(train_topics,
             train_qrels,
             test_topics,
             test_qrels)

pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    eval_metrics=["recip_rank", "map", "recall_5", "recall_10", "P.1", "P.5", "P.10", "mrt"]
)

[2023-08-02 16:53:48,564][onir_pt][DEBUG] [starting] validation
[2023-08-02 16:53:48,565][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 16:53:48,566][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/5000 s<?, ?it/s]

[2023-08-02 16:54:05,886][onir_pt][DEBUG] [finished] batches: [17.32s] [5000it] [288.68it/s]
[2023-08-02 16:54:06,279][onir_pt][DEBUG] [finished] validation [17.71s]
[2023-08-02 16:54:06,281][onir_pt][INFO] pre-validation: 0.0229
[2023-08-02 16:54:06,300][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 16:54:06,301][onir_pt][DEBUG] [starting] training
[2023-08-02 16:54:06,302][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[2023-08-02 16:54:12,457][onir_pt][DEBUG] [finished] train pairs: [6.15s] [1024it] [166.40it/s]
[2023-08-02 16:54:12,458][onir_pt][DEBUG] [finished] training [6.16s]
[2023-08-02 16:54:12,459][onir_pt][INFO] training   it=0 loss=0.2942
[2023-08-02 16:54:12,461][onir_pt][DEBUG] [starting] validation
[2023-08-02 16:54:12,462][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 16:54:12,463][onir_pt][DEBUG] [starting] batches


...

batches:   0%|          | 0/5000 s<?, ?it/s]

[2023-08-02 17:03:24,506][onir_pt][DEBUG] [finished] batches: [16.33s] [5000it] [306.19it/s]
[2023-08-02 17:03:24,881][onir_pt][DEBUG] [finished] validation [16.71s]
[2023-08-02 17:03:24,883][onir_pt][INFO] validation it=24 map=0.0533 ndcg=0.1187 P_10=0.0110
[2023-08-02 17:03:24,884][onir_pt][INFO] early stopping; model reverting back to it=4
[2023-08-02 17:26:40,031][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 17:26:40,032][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/57500 s<?, ?it/s]

[2023-08-02 17:29:43,786][onir_pt][DEBUG] [finished] batches: [03:04] [57500it] [312.92it/s]


,name,recip_rank,map,recall_5,recall_10,P.1,P.5,P.10,mrt
0,DPH,0.218014,0.192459,0.228235,0.279118,0.161765,0.053294,0.033353,361.059752
1,DPH >> VBERT,0.112653,0.100305,0.115294,0.160294,0.074706,0.026353,0.018824,465.847355


In [ ]:
# Load a version of EPIC trained on the MS-MARCO dataset
lazy_epic = onir_pt.reranker.from_checkpoint(
    'https://macavaney.us/epic.msmarco.tar.gz',
    expected_md5="2f6a16be1a6a63aab1e8fed55521a4db")

config file not found: config


[2023-08-02 14:50:47,938][onir.util.download][DEBUG] downloaded https://macavaney.us/epic.msmarco.tar.gz [5.03s] [494M] [182MB/s] [md5 hash verified]


100%|██████████| 231508/231508 s<0ms, 357751.40B/s]  
100%|██████████| 433/433 s<0ms, 497843.65B/s]
100%|██████████| 440473133/440473133 [42.34s<0ms, 10402475.15B/s]  


In [ ]:
# Use the TREC COVID dataset for this example
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [ ]:
# Build an inverted index for TREC COIVID with pyterrier
pt_index_path = './terrier_cord19'
if not os.path.exists(pt_index_path + '/data.properties'):
    indexer = pt.index.IterDictIndexer(pt_index_path)
    index_ref = indexer.index(dataset.get_corpus_iter(), fields=('abstract',), meta=('docno',))
else:
    index_ref = pt.IndexRef.of(pt_index_path + '/data.properties')
index = pt.IndexFactory.of(index_ref)

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

<ipython-input-44-bcc88b6299f5>:5: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(dataset.get_corpus_iter(), fields=('abstract',), meta=('docno',))


14:51:50.274 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
14:52:46.458 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

In [ ]:
br = pt.BatchRetrieve(index) % 30
pipeline = (br >> pt.text.get_text(dataset, 'abstract')
               >> pt.apply.generic(lambda x: x.rename(columns={'abstract': 'text'}))
               >> lazy_epic)
pt.Experiment(
    [br, pipeline],
    dataset.get_topics('title'),
    dataset.get_qrels(),
    names=['DPH', 'DPH >> EPIC (lazy)'],
    eval_metrics=["map","recip_rank", "P.5", "mrt"]
)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [458B] [3.46MB/s]
[WARNING] Download failed: Expected md5 hash to be 0307a37b6b9f1a5f233340a769d538ea but got 213064f8c4c4654f4e2d7c157b2e1fce
[INFO] [starting] https://mirror.ir-datasets.com/0307a37b6b9f1a5f233340a769d538ea
[INFO] [finished] https://mirror.ir-datasets.com/0307a37b6b9f1a5f233340a769d538ea: s] [18.7kB] [11.3MB/s]
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: s] [458B] [3.03MB/s]
[WARNING] Download failed: Expected md5 hash to be 8138424a59daea0aba751c8a891e5f54 but got 213064f8c4c4654f4e2d7c157b2e1fce
[INFO] [starting] https://mirror.ir-datasets.com/8138424a59daea0aba751c8a891e5f54
[INFO] [finished] https://mirror.ir-datasets.com/8138424a59daea0aba751c8a891e5f54: s] [1.14MB] [20.4MB/s]


[2023-08-02 14:53:00,072][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 14:53:05,648][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/375 s<?, ?it/s]

[2023-08-02 14:54:05,836][onir_pt][DEBUG] [finished] batches: [01:00] [375it] [ 6.23it/s]


,name,map,recip_rank,P.5,mrt
0,DPH,0.031769,0.766833,0.684,44.082439
1,DPH >> EPIC (lazy),0.032313,0.807889,0.724,1353.037710


In [ ]:
# vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='text', vocab_config={'train': True})

knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

# knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz', Ctext_field='text', expected_md5="d70b1d4f899690dae51161537e69ed5a")

# foo = onir_pt.reranker('knrm', 'wordvec_hash', text_field='text').fit(tr_run=train_qrels, tr_qrels=train_qrels, va_run=test_topics, va_qrels=test_qrels, tr_pairs=None)

[2023-08-02 14:54:05,984][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2023-08-02 14:54:09,732][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2023-08-02 14:54:09,738][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [3.49s] [682M] [186MB/s]
[2023-08-02 14:54:09,748][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [3.77s]
[2023-08-02 14:54:09,749][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2023-08-02 14:54:21,627][WordvecHashVocab][DEBUG] [finished] extracting vecs [11.88s]
[2023-08-02 14:54:21,629][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2023-08-02 14:56:32,577][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:11]
[2023-08-02 14:56:32,930][WordvecHashVocab][DEBUG] [starting] writing

In [ ]:
pipeline = (pt.BatchRetrieve(index) % 100 # get top 100 results
            >> pt.text.get_text(dataset, 'abstract') # fetch the document text
            >> knrm) # apply neural re-ranker

# pipeline.fit(
#     train_topics,
#     train_qrels,
#     test_topics,
#     test_qrels)

In [ ]:
br = pt.BatchRetrieve(index)
# foo = (pt.BatchRetrieve(index) # get top 100 results
#        >> pt.text.get_text(index, 'text') # fetch the document text
#        >> trained_model)
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> VBERT'],
    eval_metrics=["recip_rank", "map", "recall_5", "recall_10", "P.1", "P.5", "P.10", "mrt"]
)

[2023-08-02 14:56:56,185][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 14:56:56,470][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2023-08-02 14:56:59,203][onir_pt][DEBUG] [finished] batches: [2.73s] [1250it] [457.60it/s]


,name,recip_rank,map,recall_5,recall_10,P.1,P.5,P.10,mrt
0,DPH,0.829762,0.170652,0.007960,0.015211,0.76,0.688,0.658,53.271312
1,DPH >> VBERT,0.563130,0.054804,0.004497,0.009492,0.40,0.408,0.450,147.721786


In [ ]:
# # retrieve documents with text
# br = pt.BatchRetrieve(index, metadata=['docno', 'text'])

# # use Tf as a passage scorer on sliding window passages
# psg_scorer = (
#     pt.text.sliding(text_attr='text', length=15, prepend_attr=None)
#     >> pt.text.scorer(body_attr="text", wmodel='Tf', takes='docs')
# )

# # use psg_scorer for performing query-biased summarisation on docs retrieved by br
# retr_pipe = (br >> pt.text.snippets(psg_scorer) >> lazy_epic)

In [ ]:
# lazy_epic = onir_pt.reranker.from_checkpoint(
#     'https://macavaney.us/epic.msmarco.tar.gz',
#     expected_md5="2f6a16be1a6a63aab1e8fed55521a4db")

In [ ]:
br = pt.BatchRetrieve(index)
tf = pt.BatchRetrieve(index, wmodel="Tf")
tfidf = pt.BatchRetrieve(index, wmodel="LemurTF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

pt.Experiment(
    [tf, tfidf, bm25, pl2, br, pipeline],
    topics,
    qrels,
    names=['TF', 'TFIDF', 'BM25', 'PL2', 'BR', 'DPH'],
    eval_metrics=["recip_rank", "map", "recall_5", "recall_10", "P.1", "P.5", "P.10", "mrt"]
)

[2023-08-02 14:57:14,091][onir_pt][DEBUG] using GPU (deterministic)
[2023-08-02 14:57:14,092][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1250 s<?, ?it/s]

[2023-08-02 14:57:16,820][onir_pt][DEBUG] [finished] batches: [2.73s] [1250it] [458.22it/s]


,name,recip_rank,map,recall_5,recall_10,P.1,P.5,P.10,mrt
0,TF,0.261535,0.019708,0.001217,0.002270,0.10,0.136,0.132,39.368954
1,TFIDF,0.826500,0.192764,0.008405,0.015744,0.74,0.708,0.666,46.701783
2,BM25,0.836857,0.195498,0.008551,0.016738,0.76,0.712,0.692,43.860381
3,PL2,0.772652,0.175589,0.008581,0.015814,0.66,0.712,0.672,47.568903
4,BR,0.829762,0.170652,0.007960,0.015211,0.76,0.688,0.658,47.557327
5,DPH,0.563130,0.054804,0.004497,0.009492,0.40,0.408,0.450,99.288014


In [ ]:
# INDEX_DIR='/content/drive/MyDrive/Colab Notebooks/GH95' #diretório aonde o índice vai ficar
# indexer = pt.TRECCollectionIndexer(INDEX_DIR,
#     # vamos salvar o texto como metadados
#     meta= {'docno' : 26, 'text' : 2048},
#     # The tags from which to save the text. ELSE is special tag name, which means anything not consumed by other tags.
#     meta_tags = {'text' : 'ELSE'},
#     verbose=True,
#     overwrite=True) #para sobrescrever, caso já tenha um índice com aquele nome
# indexref = indexer.index(gh_files)
# #Indexando os arquivos -- chamando o método index no objeto TRECCollectionIndexer
# index = pt.IndexFactory.of(indexref)

In [ ]:

# INDEX_DIR='/content/drive/MyDrive/Colab Notebooks/CTRs/index' #diretório aonde o índice vai ficar
# indexer = pt.IterDictIndexer(INDEX_DIR,
#                              # vamos salvar o texto como metadados
#                              meta={'docno' : 26,
#                                    'text' : 4096,
#                                    'Intervention': 4096,
#                                    'Eligibility': 4096,
#                                    'Results': 4096,
#                                    'Adverse Events': 4096},
#                              # The tags from which to save the text. ELSE is special tag name, which means anything not consumed by other tags.
#                              verbose=True,
#                              overwrite=True) #para sobrescrever, caso já tenha um índice com aquele nome

# indexref = indexer.index(iter_ctrs())
# #Indexando os arquivos -- chamando o método index no objeto TRECCollectionIndexer
# index = pt.IndexFactory.of(indexref)

Por default, o PyTerrier aplica stemming (Porter) e remoção de stopwords (tudo para inglês). Podemos não fazer stemming (`stemmer=None`) ou fazer stemming para outro idioma  (`stemmer='portugese'` -- sim, está escrito errado) e também manter as stopwords (`stopwords=None`).


# Learning some pyterrier and information retrieval techniques:

##Index inspecting:

IndexRef represents a [IndexRef](http://terrier.org/docs/current/javadoc/org/terrier/querying/IndexRef.html) object. It can be seen as a pointer or URI that points to the location of the index file.

In [ ]:
indexref.toString()

'/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/index/data.properties'

`index.getCollectionStatistics().toString()` provides information about the index such as the number of documents indexed and the number of distinct terms.

In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 999
Number of terms: 8670
Number of postings: 193532
Number of fields: 0
Number of tokens: 440379
Field names: []
Positions:   false



In [ ]:
# check the meta-index fields
print(index.getMetaIndex().getKeys())


['docno', 'text']


In [1]:
# If necessary, we can print the entire lexicon to see the indexed terms
# for kv in index.getLexicon():
#  print("%s (%s) -> %s (%s)" % (kv.getKey(), type(kv.getKey()), kv.getValue().toString(), type(kv.getValue()) ) )

In [ ]:
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
docid = 100 #docids  start to be counted at zero
#NB: the posting list is null if the document is empty
for posting in di.getPostings(doi.getDocumentEntry(docid)):
  termid = posting.getId()
  lee = lex.getLexiconEntry(termid)
  print(lee.getKey(), end=' ')

pregnanc 60 equal 5 arm titl particip clinic analyz outcom prior intraven disord total time infect greater complet frame criteria hepat evid unit year diseas event intervent chemotherapi measur cell respons statu perform normal 1000 elig number result treatment descript advers group inclus mg type follow requir trial 8 3 2 1 iv patient id adequ cancer surgeri myocardi function cycl infarct breast need bone tumor karnofski inflammatori dai local 7 4 tissu cardiac malign renal rate carcinoma invas neutropenia lobular febril diagnosi 17 doxorubicin abnorm 13 21 marrow last primari 70 baselin patholog 08 54 site thrombocytopenia contain advanc detect stomat size vomit 15 1200 exclus previou reserv physician 62 assess second sampl cisplatin pancytopenia bidimension diarrhoea identifi 65 69 extent 26 fatigu concomit origin neutropen presenc gemcitabin feed jaundic neoplast macroscop arrest guid kp section macrophag mononuclear stroma nondescript devoid decis lobul corrobor fibroblast fibroel

In [ ]:
#List the frequencies of terms in doc 10
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
docid = 100 #docids are 0-based
#NB: postings will be null if the document is empty
for posting in di.getPostings(doi.getDocumentEntry(docid)):
  termid = posting.getId()
  lee = lex.getLexiconEntry(termid)
  print("%s with frequency %d" % (lee.getKey(),posting.getFrequency()))




pregnanc with frequency 1
60 with frequency 2
equal with frequency 1
5 with frequency 6
arm with frequency 2
titl with frequency 1
particip with frequency 2
clinic with frequency 1
analyz with frequency 1
outcom with frequency 1
prior with frequency 1
intraven with frequency 2
disord with frequency 1
total with frequency 1
time with frequency 1
infect with frequency 2
greater with frequency 1
complet with frequency 3
frame with frequency 1
criteria with frequency 2
hepat with frequency 1
evid with frequency 2
unit with frequency 2
year with frequency 1
diseas with frequency 1
event with frequency 2
intervent with frequency 2
chemotherapi with frequency 3
measur with frequency 4
cell with frequency 2
respons with frequency 3
statu with frequency 2
perform with frequency 2
normal with frequency 1
1000 with frequency 2
elig with frequency 1
number with frequency 4
result with frequency 2
treatment with frequency 1
descript with frequency 1
advers with frequency 2
group with frequency 2
in

In [ ]:
# #which documents contain the term and how often
# meta = index.getMetaIndex()
# inv = index.getInvertedIndex()
# le = lex.getLexiconEntry( "stroma" )
# # the lexicon entry is also our pointer to access the inverted index posting list
# for posting in inv.getPostings( le ):
#   docno = meta.getItem("docno", posting.getId())
#   print("%s with frequency %d " % (docno, posting.getFrequency()))

#Querying

## Iteractive Queries

###Terrier Query Language

[Fonte](http://terrier.org/docs/v1.1.1/terrier_develop.html)

Terrier offers a flexible and powerful query language for searching with phrases, fields, or specifying that terms are required to appear in the retrieved documents. Some examples of queries are the following:
* term1 term2 	retrieves documents that contains 1 or more term1 and term2 (they need not contain both)
* term1^2.3	the weight of term1 is boosted 2.3.
* +term1 +term2	retrieves documents that contain both term1 and term2.
* +term1 -term2	retrieves documents that contain term1 and do not contain term2.
* "term1 term2"	retrieves documents where the terms term1 and term2 appear in a phrase.
* "term1 term2"~n	retrieves documents where the terms term1 and term2 appear within a distance of n blocks. The order of the terms is not considered.




Várias funções de ranking implementadas (`wmodel`), incluindo  TF_IDF, PL2, DFR. A lista completa está em http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html

In [ ]:
br = pt.BatchRetrieve(index, metadata=["docno", "text"], wmodel="TF_IDF")
br.search("Patients with significantly elevated ejection fraction")

,qid,docid,docno,text,rank,score,query
0,1,493,NCT00912340,"NCT00912340 {""Clinical Trial ID"": ""NCT0091234...",0,7.837889,Patients with significantly elevated ejection ...
1,1,783,NCT01931163,"NCT01931163 {""Clinical Trial ID"": ""NCT0193116...",1,7.704695,Patients with significantly elevated ejection ...
2,1,399,NCT00698035,"NCT00698035 {""Clinical Trial ID"": ""NCT0069803...",2,7.541196,Patients with significantly elevated ejection ...
3,1,290,NCT00470847,"NCT00470847 {""Clinical Trial ID"": ""NCT0047084...",3,7.375055,Patients with significantly elevated ejection ...
4,1,711,NCT01629615,"NCT01629615 {""Clinical Trial ID"": ""NCT0162961...",4,7.314571,Patients with significantly elevated ejection ...
...,...,...,...,...,...,...,...
777,1,321,NCT00545688,"NCT00545688 {""Clinical Trial ID"": ""NCT0054568...",777,0.548988,Patients with significantly elevated ejection ...
778,1,500,NCT00929240,"NCT00929240 {""Clinical Trial ID"": ""NCT0092924...",778,0.539227,Patients with significantly elevated ejection ...
779,1,604,NCT01250379,"NCT01250379 {""Clinical Trial ID"": ""NCT0125037...",779,0.523706,Patients with significantly elevated ejection ...
780,1,846,NCT02259114,"NCT02259114 {""Clinical Trial ID"": ""NCT0225911...",780,0.521455,Patients with significantly elevated ejection ...


##Batch Queries

To assess the quality of an Information Retrieval (IR) system, it is necessary to run a significant number of queries (i.e., at least 30) and calculate evaluation metrics for them. The queries used in evaluation campaigns are commonly referred to as "topics." A topic is represented by a structure that includes an identification number, a title, a description, and a narrative. The title is a concise description of the topic. The description provides a bit more detail, and the narrative assists the individuals who produce relevance judgments in distinguishing relevant from non-relevant documents. Below, we provide an example of a topic:



```
<top>
<num> 254 </num>
<title> Earthquake Damage <title>
<desc> Find documents describing damage to property or persons caused by an earthquake and specifying the area affected. <desc>
<narr> Relevant documents will provide details on damage to buildings and material goods or injuries to people as a result of an earthquake. The geographical location (e.g. country, region, city) affected by the earthquake must also be mentioned. <narr>
</top>

```


In [ ]:
#file with topic queries
topicsFile = '/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/topics.txt'
topics = pt.io.read_topics(topicsFile)
print(topics)

      qid                                              query
0       1  all the primary trial participants do not rece...
1       2   patients with platelet count over 100 000 mm anc
2       3  heart related adverse events were recorded in ...
3       4  adult patients with histologic confirmation of...
4       5  laser therapy is in each cohort of the primary...
...   ...                                                ...
1895  196  the the primary trial intervention involves on...
1896  197  the secondary trial reported 1 single case of ...
1897  198  the secondary trial and the primary trial do n...
1898  199  the outcome measurement of the primary trial i...
1899  200  all the primary trial patients had a minimum o...

[1900 rows x 2 columns]


In [ ]:
#retrieving documents with bm25
#bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=100) #the default is to retrieve 1000 docs, but it can be changed with num_results
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
resbm25 = bm25.transform(topics)
#analyzing the list of documents retrieved for each query
resbm25

KeyboardInterrupt: ignored

#Evaluating Query Results
To evaluate how well the Information Retrieval (IR) system responded to the queries, it's necessary to have knowledge of the "expected" responses, which are known as relevance judgments. The list of documents that should have been retrieved for the set of queries from the file "topicos.txt" is found in the file "qrels.txt." A small excerpt from this file is illustrated below. The excerpt shows documents that were evaluated for query topic 4. The document "NCT01097642" was considered relevant to this query (indicated by the number 1 in the last column).

```

4 0 NCT01091974 0
4 0 NCT01094184 0
4 0 NCT01095003 0
4 0 NCT01097460 0
4 0 NCT01097642 1
4 0 NCT01104584 0
4 0 NCT01105312 0
4 0 NCT01105650 0


In [ ]:
#arquivo com os julgamentos de relevância
qrelsFile = '/content/drive/MyDrive/Colab Notebooks/CTRs/TREC/topics_qrels/qrels.txt'
qrels = pt.io.read_qrels(qrelsFile)
print(qrels)

##Evaluation metrics

There are several metrics for evaluating the quality of the results of an IR (Information Retrieval) system. Mean Average Precision (MAP) is one of the most important. Additionally, we can also look at precision at various points in the ranking, such as P@1, P@10, etc.
To list the implemented metrics, use `ir_measures.parse_trec_measure('official') `

In [ ]:
from pyterrier.measures import *
# Evaluating the query results in terms of MAP, P@1, P@5, and P@10.
pt.Utils.evaluate(resbm25, qrels, metrics = ['map','P_1','P_5','P_10']) #mean for all queries


In [ ]:
#visualizing the result of MAP per query.
pt.Utils.evaluate(resbm25, qrels, metrics = ['map'], perquery=True)

## Precision/Recall Plots

Generating the interpolated precision values ​​at standard recall levels to be able to draw the precision-recall curve

In [ ]:
iprec = pt.Utils.evaluate(resbm25, qrels, metrics = [IPrec@0.0,IPrec@0.1,IPrec@0.2,IPrec@0.3,IPrec@0.4,IPrec@0.5,IPrec@0.6,IPrec@0.7,IPrec@0.8,IPrec@0.9,IPrec@1.0])
iprec.values()

In [ ]:
import matplotlib.pyplot as plt
x=[0, 0.1, 0.2,0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y=iprec.values()
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('recall')
plt.ylabel('precision')
plt.title("Curva de precisão e revocação")
plt.plot(x,y,'bo-', linewidth=2)
plt.show()


#Conducting comparative experiments
Knowing the results of metrics for an experimental configuration alone is not sufficient. In most cases, the goal is to compare different configurations. In this case, we will use pt.Experiment() to compare 4 ranking functions: Tf, TF-IDF, BM25, and PL2.
For more details, visit https://pyterrier.readthedocs.io/en/latest/experiments.html.

In [ ]:
tf = pt.BatchRetrieve(index, wmodel="Tf")
tfidf = pt.BatchRetrieve(index, wmodel="LemurTF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
pl2 = pt.BatchRetrieve(index, wmodel="PL2")

#Analyzing the MAP values for the query set
pt.Experiment([tf, tfidf, bm25, pl2], topics, qrels, eval_metrics=["map"],perquery=False,round=4)